<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/fixed_income/spot_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np

# compute bond price from maturity, coupon, and yield,
#      or from spot rates
def price(**kwargs):
    n = int(2 * kwargs['maturity'])
    c = 100 * kwargs['coupon'] / 2
    if 'yld' in kwargs.keys():
        yld = kwargs['yld'] / 2
        return c * np.sum((1 + yld) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld) ** n
    else:
        spots = np.array(kwargs['spots']) / 2
        return c * np.sum((1 + spots[:n]) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + spots[n - 1]) ** n

def error(bond, spots):
    p = price(**bond)
    m = bond['maturity']
    n = int(2 * m)
    phat = price(maturity=m, coupon=bond['coupon'], spots=spots[:n])
    return np.log(phat / p)

# compute forward rates from spot rates
def forward_rates(spots):
    future_factors = (1+spots/2)**np.arange(1,len(spots)+1)
    change_logs = np.diff(np.log(future_factors))
    f = 2 * (np.exp(change_logs)-1)
    return np.concatenate(([spots[0]], f))

def objective(bonds, spots, weight_on_smooth_forwards=0.5):
    # --------------
    # Parameters:
    #   bonds - the input bond data
    #   spots - the spot rates as the parameters of the objective function
    #   weight_on_smooth_forwards - The bond prices may have measurement errors.
    #                               We impose some "rationality" on the prices by requiring smoothly varying forward rates.
    # --------------

    # the difference between theoretical price computed by spot rates and the real price of bonds
    sse = np.sum([error(bond, spots)**2 for bond in bonds])
    forwards = forward_rates(spots)
    # penalize not smooth forward rates.
    diffs = np.sum(np.diff(np.log(1+forwards/2))**2)
    return sse + weight_on_smooth_forwards*diffs

def spot_rates(bonds):
    maturities = [bond['maturity'] for bond in bonds]
    n = int(2*np.max(maturities))

    # minimize the objective function defined above
    result = minimize(lambda x: objective(bonds, x),    # objective function
                      [0.05]*n                          # initial guess
                      )

    return result.x if result.success==True else np.nan

In [2]:
# example bonds data

bonds=[
        {"maturity": 2, "coupon": 0.04, "yld": 0.02},
        {"maturity": 4, "coupon": 0.04, "yld": 0.03},
        {"maturity": 6, "coupon": 0.04, "yld": 0.04},
        {"maturity": 8, "coupon": 0.04, "yld": 0.05},
        {"maturity": 10, "coupon": 0.04, "yld": 0.06},
        ]

# spot rates
spots = spot_rates(bonds)

# forward rates
forwards = forward_rates(spots)

df = pd.DataFrame([spots, forwards]).transpose()
df = df.reset_index()
df['index'] = df['index'] * 0.5 + 0.5
df.rename(columns={'index' : 'Maturities', 0 : 'spot rates', 1: 'forward rates'})

,Maturities,spot rates,forward rates
0,0.5,0.016403,0.016403
1,1.0,0.017191,0.017980
2,1.5,0.018497,0.021110
3,2.0,0.020314,0.025777
4,2.5,0.022639,0.031966
5,3.0,0.025181,0.037936
6,3.5,0.027816,0.043697
7,4.0,0.030483,0.049255
8,4.5,0.033151,0.054618
9,5.0,0.035830,0.060099
